<a href="https://colab.research.google.com/github/MdelaVilla/MORS/blob/main/MORS_Sesion_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: conectar con drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
file = ('/content/drive/MyDrive/Colab Notebooks/MORS/public_development_en/train_en.tsv')
fileEval = ('/content/drive/MyDrive/Colab Notebooks/MORS/public_development_en/dev_en.tsv')

In [ ]:
data = pd.read_csv(file, delimiter='\t', usecols =["text","HS"])
evalEtiquetas = pd.read_csv(fileEval, delimiter='\t', usecols =["HS"])
eval = pd.read_csv(fileEval, delimiter='\t', usecols =["text"])

In [ ]:
data.head()

,text,HS
0,"Hurray, saving us $$$ in so many ways @potus @...",1
1,Why would young fighting age men be the vast m...,1
2,@KamalaHarris Illegals Dump their Kids at the ...,1
3,NY Times: 'Nearly All White' States Pose 'an A...,0
4,Orban in Brussels: European leaders are ignori...,0


In [ ]:
data.size

18000

In [ ]:
eval.head()

,text
0,I swear I’m getting to places just in the nick...
1,I’m an immigrant — and Trump is right on immig...
2,#IllegalImmigrants #IllegalAliens #ElectoralSy...
3,@DRUDGE_REPORT We have our own invasion issues...
4,Worker Charged With Sexually Molesting Eight C...


In [ ]:
eval.size

1000

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
labels = data.HS
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 3), stop_words='english')
features = tfidf.fit_transform(data.text)

print(features.shape)

print(features.toarray())

(9000, 4380)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
print(tfidf.idf_)

[5.46070006 7.90786638 8.15918081 ... 8.15918081 6.84699442 8.15918081]


In [ ]:
from sklearn.model_selection import train_test_split

# división del conjunto en entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(features, labels,

                                                    stratify=labels,

                                                    test_size=0.2,

                                                    random_state=1234)



In [ ]:
from sklearn.ensemble import RandomForestClassifier

#from sklearn import svm

#rom sklearn.neural_network import MLPClassifier

clasificador = RandomForestClassifier(criterion='entropy', n_estimators=100, class_weight={1: 0.6})

#clasificador = svm.SVC(kernel='rbf', gamma=0.001, C=300, class_weight={0: 0.7})

#clasificador = MLPClassifier()

In [ ]:
clasificador.fit(X_train, y_train)

pred_y = clasificador.predict(X_test)

print("CCR: %f"%(clasificador.score(X_test, y_test)))

#CCR: 0.xxxxxx  Random Forest
#CCR: 0.xxxxxx  SVM
#CCR: 0.xxxxxx


CCR: 0.796111


In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, pred_y)

array([[913, 130],
       [237, 520]])

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, pred_y))

              precision    recall  f1-score   support

           0       0.79      0.88      0.83      1043
           1       0.80      0.69      0.74       757

    accuracy                           0.80      1800
   macro avg       0.80      0.78      0.79      1800
weighted avg       0.80      0.80      0.79      1800



In [ ]:
evalConTfidf = tfidf.transform(eval.text)
print(evalConTfidf.shape)
evalPredict = clasificador.predict(evalConTfidf)
print("CCR: %f"%(clasificador.score(evalConTfidf, evalEtiquetas)))
print(confusion_matrix(evalEtiquetas, evalPredict))
print(classification_report(evalEtiquetas, evalPredict))
print(evalPredict)

(1000, 4380)
CCR: 0.751000
[[462 111]
 [138 289]]
              precision    recall  f1-score   support

           0       0.77      0.81      0.79       573
           1       0.72      0.68      0.70       427

    accuracy                           0.75      1000
   macro avg       0.75      0.74      0.74      1000
weighted avg       0.75      0.75      0.75      1000

[0 0 0 1 0 0 1 0 0 0 1 1 1 0 0 1 0 1 0 0 0 1 0 1 0 0 0 1 0 1 0 1 0 1 0 1 0
 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 1 0 1 0 0 1 1 0 0 1 0 0 0
 0 1 0 0 0 1 1 1 1 1 0 1 0 1 0 0 0 0 0 1 1 1 0 1 0 0 1 1 1 0 0 1 0 0 1 0 0
 1 0 0 1 1 0 1 0 1 0 0 0 1 1 0 1 1 0 0 1 0 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0
 0 0 0 1 1 1 0 0 0 0 0 1 1 1 0 0 1 1 1 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 1 0 1
 0 1 1 1 0 0 1 0 0 1 0 1 0 0 0 1 1 0 0 1 0 1 0 0 1 0 0 1 0 0 1 0 1 0 1 0 1
 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 1 0 0 1 0
 1 0 0 1 0 1 0 0 1 0 1 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 1 1 0 

In [ ]:
print(evalConTfidf)

  (0, 399)	0.2925220925528835
  (0, 720)	0.32284368438178607
  (0, 892)	0.2925220925528835
  (0, 1578)	0.2307247746236995
  (0, 1771)	0.1824973737167268
  (0, 1813)	0.0709850653233802
  (0, 2101)	0.15531141265648518
  (0, 2258)	0.28215053316419186
  (0, 2481)	0.15578282219933468
  (0, 2857)	0.28215053316419186
  (0, 3141)	0.13582321973861758
  (0, 3174)	0.2696740898824151
  (0, 3304)	0.2196647816507973
  (0, 3767)	0.31386740315068906
  (0, 3876)	0.17213279385217714
  (0, 3905)	0.33441612361322876
  (0, 4195)	0.2145844287140572
  (1, 1813)	0.1833972342779116
  (1, 1905)	0.21719628616764827
  (1, 1943)	0.22308118737156343
  (1, 1948)	0.35691548650183447
  (1, 3275)	0.2880944677948355
  (1, 3687)	0.3612706676126444
  (1, 3688)	0.4854030677348679
  (1, 3944)	0.2520452224725195
  :	:
  (997, 3814)	0.22721353659953064
  (997, 4214)	0.1670117727660987
  (997, 4261)	0.19228877237105751
  (998, 438)	0.17448462320343153
  (998, 455)	0.4405682802226303
  (998, 471)	0.3059020964296126
  (998, 1319